In [246]:
import pandas as pd

df = pd.read_excel('../数据/MCMB数据.xlsx')
df.query('年 == 2000')
# print(test[test.index[0]])

,年,人口密度（人/平方千米）,耕地面积,森林面积,大象致人伤亡数量,食草动物致人伤亡数量,食肉动物致人伤亡数量,营地累计数量和住宿（MMNR内部以及周边地区）,犀牛栖息地百分比,牛放牧扩大占比
52,2000,65.02,112797.0,26981.0,4,1,8,23,18.0,17.0


In [241]:
from scipy.optimize import minimize
import numpy as np
import math


def getFun(args):
    k1, k2, k3 = args
    fun1 = lambda x: k3 * math.exp(x[2])
    fun2 = lambda x: (k1 * math.exp(x[0] / 1000) + k2 * math.exp(x[1] / 1000)) - (k3 * math.exp(x[2] / 1000))

    f = lambda x: -(fun1(x) / (fun1(x) + fun2(x)) - fun2(x) / (fun1(x) + fun2(x)))
    return f


def getCon(args):
    argsKey = list(args.keys())
    x1min = 0
    x1max = args['第2000年人口']

    x2min = 0
    x2max = args['第2000年耕地']

    x3min = 0
    x3max = args['第2000年森林']
    cons = (
        {'type': 'ineq', 'fun': lambda x: x[0] - x1min},
        {'type': 'ineq', 'fun': lambda x: -x[0] + x1max},

        {'type': 'ineq', 'fun': lambda x: x[1] - x2min},
        {'type': 'ineq', 'fun': lambda x: -x[1] + x2max},

        {'type': 'ineq', 'fun': lambda x: x[2] - x3min},
        {'type': 'ineq', 'fun': lambda x: -x[2] + x3max},

        {'type': 'ineq', 'fun': lambda x: (args['第2000年耕地'] - x[1]) / (args['第2000年人口'] - x[0] + 1) -
                                          args['sum1948-2000(bi/ai)'] / (2000 - 1948 + 1)},
    )
    return cons


# k1, k2, k3
argsK = (1, 1, 1)

search2000 = df.query('年 == 2000')
year2000 = search2000.loc[search2000.index.values[0]].to_dict()
sum19482000biai = 0
for i, row in df.iterrows():
    sum19482000biai += row['耕地面积'] / row['人口密度（人/平方千米）']

argsCon = {
    '第2000年人口': year2000['人口密度（人/平方千米）'],
    '第2000年耕地': year2000['耕地面积'],
    '第2000年森林': year2000['森林面积'],
    'sum1948-2000(bi/ai)': sum19482000biai
}
print(argsCon)

# 变量初始值
x0 = np.asarray((1, 1, 1))
res = minimize(getFun(argsK), x0, constraints=getCon(argsCon))

print(res)

'''
x1: 人口减少
x2: 耕地减少
x3: 森林增加
'''

{'第2000年人口': 65.02, '第2000年耕地': 112797.0, '第2000年森林': 26981.0, 'sum1948-2000(bi/ai)': 94926.16360342082}
     fun: -0.9999987130529299
     jac: array([ 0.00000000e+00,  0.00000000e+00, -1.28895044e-06])
 message: 'Optimization terminated successfully'
    nfev: 80
     nit: 20
    njev: 20
  status: 0
 success: True
       x: array([ 9.81366218,  0.98185156, 14.2528675 ])


# 第一问区域A求解结果
x1 = 9.81
x2 = 0.98
x3 = 14.25

因此针对于2000年：

对于A区域，应该采取如下方案：
1. 减少9.81%的人口密度
2. 减少0.98%的耕地面积
3. 增加14.25%的森林面积